In [121]:
import qiskit
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit import Aer, transpile, assemble
import cmath
import random
import numpy as np
from scipy.optimize import minimize
from qiskit.quantum_info.operators import Operator, Pauli
from qiskit.extensions import UnitaryGate



In [122]:
def convert_unitary_to_operator(theta):
    mat = [
        [cmath.cos(theta), -cmath.sin(theta)],
        [cmath.sin(theta), cmath.cos(theta)]
    ]
    return Operator(mat)

In [123]:
def convert_unitary_to_gate(theta):
    mat = [
        [cmath.cos(theta), -cmath.sin(theta)],
        [cmath.sin(theta), cmath.cos(theta)]
    ]
    return UnitaryGate(mat)

In [124]:
convert_unitary_to_gate(np.pi/4)

Instruction(name='unitary', num_qubits=1, num_clbits=0, params=[array([[ 0.70710678-0.j, -0.70710678-0.j],
       [ 0.70710678+0.j,  0.70710678-0.j]])])

In [89]:
def apply_fixed_ansatz(qubits, parameters):

    for iz in range (0, len(qubits)):
        circ.ry(parameters[0][iz], qubits[iz])

    circ.cz(qubits[0], qubits[1])
    circ.cz(qubits[2], qubits[0])

    for iz in range (0, len(qubits)):
        circ.ry(parameters[1][iz], qubits[iz])

    circ.cz(qubits[1], qubits[2])
    circ.cz(qubits[2], qubits[0])

    for iz in range (0, len(qubits)):
        circ.ry(parameters[2][iz], qubits[iz])

circ = QuantumCircuit(3)
apply_fixed_ansatz([0, 1, 2], [ [1, 1, 1], [1, 1, 1], [1, 1, 1] ])
circ.draw()

┌───────┐      ┌───────┐      ┌───────┐
q_0: ┤ Ry(1) ├─■──■─┤ Ry(1) ├────■─┤ Ry(1) ├
     ├───────┤ │  │ ├───────┤    │ ├───────┤
q_1: ┤ Ry(1) ├─■──┼─┤ Ry(1) ├─■──┼─┤ Ry(1) ├
     ├───────┤    │ ├───────┤ │  │ ├───────┤
q_2: ┤ Ry(1) ├────■─┤ Ry(1) ├─■──■─┤ Ry(1) ├
     └───────┘      └───────┘      └───────┘

In [99]:
# Creates the Hadamard test

def had_test(gate_type, qubits, auxiliary_index, parameters):

    circ.h(auxiliary_index)

    apply_fixed_ansatz(qubits, parameters)

    for first in range (0, len(gate_type)):
        for ie in range (0, len(gate_type[0])):
            if (gate_type[first][ie] == 1):
                circ.cz(auxiliary_index, qubits[ie])
            elif (gate_type[first][ie] == 2):
                # print(2)
                circ.cx(auxiliary_index, qubits[ie])
            elif (gate_type[first][ie] == 3):
                circ.cy(auxiliary_index, qubits[ie])
    
    circ.h(auxiliary_index)
    
circ = QuantumCircuit(4)
had_test([ [0, 0, 0], [0, 0, 1] ], [1, 2, 3], 0, [ [1, 1, 1], [1, 1, 1], [1, 1, 1] ])
circ.draw()

┌───┐                                   ┌───┐
q_0: ──┤ H ├─────────────────────────────────■─┤ H ├
     ┌─┴───┴─┐      ┌───────┐      ┌───────┐ │ └───┘
q_1: ┤ Ry(1) ├─■──■─┤ Ry(1) ├────■─┤ Ry(1) ├─┼──────
     ├───────┤ │  │ ├───────┤    │ ├───────┤ │      
q_2: ┤ Ry(1) ├─■──┼─┤ Ry(1) ├─■──┼─┤ Ry(1) ├─┼──────
     ├───────┤    │ ├───────┤ │  │ ├───────┤ │      
q_3: ┤ Ry(1) ├────■─┤ Ry(1) ├─■──■─┤ Ry(1) ├─■──────
     └───────┘      └───────┘      └───────┘

In [100]:
# Creates controlled anstaz for calculating |<b|psi>|^2 with a Hadamard test

def control_fixed_ansatz(qubits, parameters, auxiliary, reg):

    for i in range (0, len(qubits)):
        circ.cry(parameters[0][i], qiskit.circuit.Qubit(reg, auxiliary), qiskit.circuit.Qubit(reg, qubits[i]))

    circ.ccx(auxiliary, qubits[1], 4)
    circ.cz(qubits[0], 4)
    circ.ccx(auxiliary, qubits[1], 4)

    circ.ccx(auxiliary, qubits[0], 4)
    circ.cz(qubits[2], 4)
    circ.ccx(auxiliary, qubits[0], 4)

    for i in range (0, len(qubits)):
        circ.cry(parameters[1][i], qiskit.circuit.Qubit(reg, auxiliary), qiskit.circuit.Qubit(reg, qubits[i]))

    circ.ccx(auxiliary, qubits[2], 4)
    circ.cz(qubits[1], 4)
    circ.ccx(auxiliary, qubits[2], 4)

    circ.ccx(auxiliary, qubits[0], 4)
    circ.cz(qubits[2], 4)
    circ.ccx(auxiliary, qubits[0], 4)

    for i in range (0, len(qubits)):
        circ.cry(parameters[2][i], qiskit.circuit.Qubit(reg, auxiliary), qiskit.circuit.Qubit(reg, qubits[i]))

q_reg = QuantumRegister(5)
circ = QuantumCircuit(q_reg)
control_fixed_ansatz([1, 2, 3], [ [1, 1, 1], [1, 1, 1], [1, 1, 1] ], 0, q_reg)
circ.draw()

»
q228268_0: ────■────────■────────■──────■───────■────■───────■──────■────»
           ┌───┴───┐    │        │      │       │    │       │  ┌───┴───┐»
q228268_1: ┤ Ry(1) ├────┼────────┼──────┼───■───┼────■───────■──┤ Ry(1) ├»
           └───────┘┌───┴───┐    │      │   │   │    │       │  └───────┘»
q228268_2: ─────────┤ Ry(1) ├────┼──────■───┼───■────┼───────┼───────────»
                    └───────┘┌───┴───┐  │   │   │    │       │           »
q228268_3: ──────────────────┤ Ry(1) ├──┼───┼───┼────┼───■───┼───────────»
                             └───────┘┌─┴─┐ │ ┌─┴─┐┌─┴─┐ │ ┌─┴─┐         »
q228268_4: ───────────────────────────┤ X ├─■─┤ X ├┤ X ├─■─┤ X ├─────────»
                                      └───┘   └───┘└───┘   └───┘         »
«                                                                         »
«q228268_0: ────■────────■──────■───────■────■───────■──────■────────■────»
«               │        │      │       │    │       │  ┌───┴───┐    │    »
«q228268_1: ────┼────────┼──────┼───────┼────■───────■──┤ Ry(1) ├────┼────»
«           ┌───┴───┐    │      │       │    │       │  └───────┘┌───┴───┐»
«q228268_2: ┤ Ry(1) ├────┼──────┼───■───┼────┼───────┼───────────┤ Ry(1) ├»
«           └───────┘┌───┴───┐  │   │   │    │       │           └───────┘»
«q228268_3: ─────────┤ Ry(1) ├──■───┼───■────┼───■───┼────────────────────»
«                    └───────┘┌─┴─┐ │ ┌─┴─┐┌─┴─┐ │ ┌─┴─┐                  »
«q228268_4: ──────────────────┤ X ├─■─┤ X ├┤ X ├─■─┤ X ├──────────────────»
«                             └───┘   └───┘└───┘   └───┘                  »
«                    
«q228268_0: ────■────
«               │    
«q228268_1: ────┼────
«               │    
«q228268_2: ────┼────
«           ┌───┴───┐
«q228268_3: ┤ Ry(1) ├
«           └───────┘
«q228268_4: ─────────
«

In [129]:
u = convert_unitary_to_gate(np.pi/4)
def control_b(auxiliary, qubits):
    # u_circuit = QuantumCircuit(1, name='u')
    # u_circuit.append(u)

    # bs_gate = u_circuit.to_gate()

    # print(bs_gate)
    cb = u.control(1)

    circ.append(cb, [auxiliary, 1])

    # for ia in qubits:
    #     circ.ch(auxiliary, ia)

circ = QuantumCircuit(4)
control_b(0, [1, 2, 3])
circ.draw()

/Users/lucyjiao/miniforge3/envs/quantum/lib/python3.11/site-packages/numpy/linalg/linalg.py:2154: RuntimeWarning: divide by zero encountered in det
  r = _umath_linalg.det(a, signature=signature)
/Users/lucyjiao/miniforge3/envs/quantum/lib/python3.11/site-packages/numpy/linalg/linalg.py:2154: RuntimeWarning: invalid value encountered in det
  r = _umath_linalg.det(a, signature=signature)


q_0: ─────■─────
     ┌────┴────┐
q_1: ┤ Unitary ├
     └─────────┘
q_2: ───────────
                
q_3: ───────────

In [130]:
# Create the controlled Hadamard test, for calculating <psi|psi>

def special_had_test(gate_type, qubits, auxiliary_index, parameters, reg):

    circ.h(auxiliary_index)

    control_fixed_ansatz(qubits, parameters, auxiliary_index, reg)

    for ty in range (0, len(gate_type)):
        if (gate_type[ty] == 1):
            circ.cz(auxiliary_index, qubits[ty])
        elif (gate_type[ty] == 2):
            circ.cx(auxiliary_index, qubits[ty])
        elif (gate_type[ty] == 3):
            circ.cy(auxiliary_index, qubits[ty])


    control_b(auxiliary_index, qubits)
    
    circ.h(auxiliary_index)

q_reg = QuantumRegister(5)
circ = QuantumCircuit(q_reg)
special_had_test([ [0, 0, 0], [0, 0, 1] ], [1, 2, 3], 0, [ [1, 1, 1], [1, 1, 1], [1, 1, 1] ], q_reg)
circ.draw()

┌───┐                                                              »
q243533_0: ┤ H ├────■────────■────────■──────■───────■────■───────■──────■────»
           └───┘┌───┴───┐    │        │      │       │    │       │  ┌───┴───┐»
q243533_1: ─────┤ Ry(1) ├────┼────────┼──────┼───■───┼────■───────■──┤ Ry(1) ├»
                └───────┘┌───┴───┐    │      │   │   │    │       │  └───────┘»
q243533_2: ──────────────┤ Ry(1) ├────┼──────■───┼───■────┼───────┼───────────»
                         └───────┘┌───┴───┐  │   │   │    │       │           »
q243533_3: ───────────────────────┤ Ry(1) ├──┼───┼───┼────┼───■───┼───────────»
                                  └───────┘┌─┴─┐ │ ┌─┴─┐┌─┴─┐ │ ┌─┴─┐         »
q243533_4: ────────────────────────────────┤ X ├─■─┤ X ├┤ X ├─■─┤ X ├─────────»
                                           └───┘   └───┘└───┘   └───┘         »
«                                                                         »
«q243533_0: ────■────────■──────■───────■────■───────■──────■────────■────»
«               │        │      │       │    │       │  ┌───┴───┐    │    »
«q243533_1: ────┼────────┼──────┼───────┼────■───────■──┤ Ry(1) ├────┼────»
«           ┌───┴───┐    │      │       │    │       │  └───────┘┌───┴───┐»
«q243533_2: ┤ Ry(1) ├────┼──────┼───■───┼────┼───────┼───────────┤ Ry(1) ├»
«           └───────┘┌───┴───┐  │   │   │    │       │           └───────┘»
«q243533_3: ─────────┤ Ry(1) ├──■───┼───■────┼───■───┼────────────────────»
«                    └───────┘┌─┴─┐ │ ┌─┴─┐┌─┴─┐ │ ┌─┴─┐                  »
«q243533_4: ──────────────────┤ X ├─■─┤ X ├┤ X ├─■─┤ X ├──────────────────»
«                             └───┘   └───┘└───┘   └───┘                  »
«                               ┌───┐
«q243533_0: ────■─────────■─────┤ H ├
«               │    ┌────┴────┐└───┘
«q243533_1: ────┼────┤ Unitary ├─────
«               │    └─────────┘     
«q243533_2: ────┼────────────────────
«           ┌───┴───┐                
«q243533_3: ┤ Ry(1) ├────────────────
«           └───────┘                
«q243533_4: ─────────────────────────
«

## Simulation

In [103]:
coefficient_set = [0.55, 0.45]
gate_set = [ [0, 0, 0], [0, 0, 1] ]

# coefficient_set = [0.55, 0.45]
# gate_set = [ [0], [1] ]

# Implements the entire cost function on the quantum circuit

def calculate_cost_function(parameters):
    
    global opt

    overall_sum_1 = 0
    
    parameters = [parameters[0:3], parameters[3:6], parameters[6:9] ]

    for i in range(0, len(gate_set)):
        for j in range(0, len(gate_set)):

            global circ

            qctl = QuantumRegister(5)
            qc = ClassicalRegister(5)
            circ = QuantumCircuit(qctl, qc)

            backend = Aer.get_backend('aer_simulator')
            
            multiply = coefficient_set[i]*coefficient_set[j]

            had_test([gate_set[i], gate_set[j] ], [1, 2, 3], 0, parameters)

            circ.save_statevector()
            t_circ = transpile(circ, backend)
            # qobj = assemble(t_circ)
            job = backend.run(t_circ)

            result = job.result()
            outputstate = np.real(result.get_statevector(circ, decimals=100))
            o = outputstate

            m_sum = 0
            for l in range (0, len(o)):
                if (l%2 == 1):
                    n = o[l]**2
                    m_sum+=n

            overall_sum_1+=multiply*(1-(2*m_sum))

    overall_sum_2 = 0

    for i in range(0, len(gate_set)):
        for j in range(0, len(gate_set)):

            multiply = coefficient_set[i]*coefficient_set[j]
            mult = 1

            for extra in range(0, 2):

                qctl = QuantumRegister(5)
                qc = ClassicalRegister(5)
                circ = QuantumCircuit(qctl, qc)

                backend = Aer.get_backend('aer_simulator')

                if (extra == 0):
                    special_had_test(gate_set[i], [1, 2, 3], 0, parameters, qctl)
                if (extra == 1):
                    special_had_test(gate_set[j], [1, 2, 3], 0, parameters, qctl)

                circ.save_statevector()    
                t_circ = transpile(circ, backend)
                # qobj = assemble(t_circ)
                job = backend.run(t_circ)

                result = job.result()
                outputstate = np.real(result.get_statevector(circ, decimals=100))
                o = outputstate

                m_sum = 0
                for l in range (0, len(o)):
                    if (l%2 == 1):
                        n = o[l]**2
                        m_sum+=n
                mult = mult*(1-(2*m_sum))

            overall_sum_2+=multiply*mult
            
    # print(1-float(overall_sum_2/overall_sum_1))

    return 1-float(overall_sum_2/overall_sum_1)

In [8]:
from qiskit import IBMQ
# load IBM account
IBMQ.save_account('2bb24b1ffb16645433661cd2214aedc8d53fef6a635ac74857b0e282de4e4597754228444daece952006fb5dff87b434835547e926a5539b24f800818c08e394',overwrite=True)
IBMQ.load_account()

provider = IBMQ.providers()
provider = IBMQ.get_provider(hub='ibm-q-education', group='harvard', project='qse-210')

/var/folders/07/x8mtl3_53h95v_q_0ddbc_fh0000gn/T/ipykernel_33058/2447371092.py:3: DeprecationWarning: The package qiskit.providers.ibmq is being deprecated. Please see https://ibm.biz/provider_migration_guide to get instructions on how to migrate to qiskit-ibm-provider (https://github.com/Qiskit/qiskit-ibm-provider) and qiskit-ibm-runtime (https://github.com/Qiskit/qiskit-ibm-runtime).
  IBMQ.save_account('2bb24b1ffb16645433661cd2214aedc8d53fef6a635ac74857b0e282de4e4597754228444daece952006fb5dff87b434835547e926a5539b24f800818c08e394',overwrite=True)
/var/folders/07/x8mtl3_53h95v_q_0ddbc_fh0000gn/T/ipykernel_33058/2447371092.py:3: DeprecationWarning: The qiskit.IBMQ entrypoint and the qiskit-ibmq-provider package (accessible from 'qiskit.providers.ibmq`) are deprecated and will be removed in a future release. Instead you should use the qiskit-ibm-provider package which is accessible from 'qiskit_ibm_provider'. You can install it with 'pip install qiskit_ibm_provider'. Just replace 'qisk

In [10]:
backend=provider.get_backend("ibm_perth")
# backend=provider.get_backend("ibmq_quito")

In [104]:


out = minimize(calculate_cost_function, x0=[float(random.randint(0,3000))/1000 for i in range(0, 9)], method="COBYLA", options={'maxiter':200})
print(out)

out_f = [out['x'][0:3], out['x'][3:6], out['x'][6:9] ]



 message: Maximum number of function evaluations has been exceeded.
 success: False
  status: 2
     fun: 0.01459525103224757
       x: [ 8.790e-01  3.240e+00  1.690e+00  1.380e+00  3.106e+00
            1.335e+00  2.978e+00  1.916e+00  6.716e-01]
    nfev: 200
   maxcv: 0.0


In [105]:
circ = QuantumCircuit(3, 3)
apply_fixed_ansatz([0, 1, 2], out_f)
circ.save_statevector()

backend = Aer.get_backend('aer_simulator')
t_circ = transpile(circ, backend)
print(circ)

job = backend.run(t_circ, shots=8192)

result = job.result()
print(result.get_counts())
o = result.get_statevector(circ, decimals=10)
print(o)

# a1 = coefficient_set[1]*np.array([ [1,0], [0,-1] ])
# a2 = coefficient_set[0]*np.array([ [1,0], [0,1]])
a1 = coefficient_set[1]*np.array([ [1,0,0,0,0,0,0,0], [0,1,0,0,0,0,0,0], [0,0,1,0,0,0,0,0], [0,0,0,1,0,0,0,0], [0,0,0,0,-1,0,0,0], [0,0,0,0,0,-1,0,0], [0,0,0,0,0,0,-1,0], [0,0,0,0,0,0,0,-1] ])
a2 = coefficient_set[0]*np.array([ [1,0,0,0,0,0,0,0], [0,1,0,0,0,0,0,0], [0,0,1,0,0,0,0,0], [0,0,0,1,0,0,0,0], [0,0,0,0,1,0,0,0], [0,0,0,0,0,1,0,0], [0,0,0,0,0,0,1,0], [0,0,0,0,0,0,0,1] ])
# a2x = coefficient_set[0]*np.array([ [0,1,0,0,0,0,0,0], [1,0,0,0,0,0,0,0], [0,0,0,1,0,0,0,0], [0,0,1,0,0,0,0,0], [0,0,0,0,0,1,0,0], [0,0,0,0,1,0,0,0], [0,0,0,0,0,0,0,1], [0,0,0,0,0,0,1,0] ])

a3 = np.add(a2, a1)

# b = np.array([float(1/np.sqrt(2)), float(1/np.sqrt(2))])

b = np.array([float(1/np.sqrt(8)),float(1/np.sqrt(8)),float(1/np.sqrt(8)),float(1/np.sqrt(8)),float(1/np.sqrt(8)),float(1/np.sqrt(8)),float(1/np.sqrt(8)),float(1/np.sqrt(8))])

print((b.dot(a3.dot(o)/(np.linalg.norm(a3.dot(o))))))



     ┌─────────────┐      ┌────────────┐       ┌────────────┐ statevector 
q_0: ┤ Ry(0.87898) ├─■──■─┤ Ry(1.3804) ├────■──┤ Ry(2.9777) ├──────░──────
     └┬────────────┤ │  │ ├────────────┤    │  ├────────────┤      ░      
q_1: ─┤ Ry(3.2404) ├─■──┼─┤ Ry(3.1064) ├─■──┼──┤ Ry(1.9155) ├──────░──────
      ├────────────┤    │ ├────────────┤ │  │ ┌┴────────────┤      ░      
q_2: ─┤ Ry(1.6895) ├────■─┤ Ry(1.3351) ├─■──■─┤ Ry(0.67157) ├──────░──────
      └────────────┘      └────────────┘      └─────────────┘      ░      
c: 3/═════════════════════════════════════════════════════════════════════
                                                                          
{'000': 8192}
Statevector([-0.05435734+0.j, -0.04535329+0.j, -0.04735666+0.j,
             -0.05423474+0.j, -0.44979095+0.j, -0.3990691 +0.j,
             -0.59870492+0.j, -0.51941186+0.j],
            dims=(2, 2, 2))
(-0.9926755507160081+0j)


In [50]:
circ = QuantumCircuit(3, 3)
apply_fixed_ansatz([0, 1, 2], out_f)
circ.save_statevector()
# circ.measure_all(add_bits=False)

backend = Aer.get_backend('aer_simulator')
t_circ = transpile(circ, backend)
# qobj = assemble(t_circ)

print(circ)


      ┌────────────┐  statevector 
q_0: ─┤ Ry(2.1747) ├───────░──────
     ┌┴────────────┴┐      ░      
q_1: ┤ Ry(0.027041) ├──────░──────
     └┬───────────┬─┘      ░      
q_2: ─┤ Ry(3.142) ├────────░──────
      └───────────┘        ░      
c: 3/═════════════════════════════
                                  


In [52]:
# backend = Aer.get_backend('aer_simulator')


job = backend.run(t_circ, shots=8192)

result = job.result()
print(result.get_counts())
o = result.get_statevector(circ, decimals=10)

# a1 = coefficient_set[1]*np.array([ [1,0,0,0,0,0,0,0], [0,1,0,0,0,0,0,0], [0,0,1,0,0,0,0,0], [0,0,0,1,0,0,0,0], [0,0,0,0,-1,0,0,0], [0,0,0,0,0,-1,0,0], [0,0,0,0,0,0,-1,0], [0,0,0,0,0,0,0,-1] ])
a2 = coefficient_set[0]*np.array([ [1,0,0,0,0,0,0,0], [0,1,0,0,0,0,0,0], [0,0,1,0,0,0,0,0], [0,0,0,1,0,0,0,0], [0,0,0,0,1,0,0,0], [0,0,0,0,0,1,0,0], [0,0,0,0,0,0,1,0], [0,0,0,0,0,0,0,1] ])
a2x = coefficient_set[0]*np.array([ [0,1,0,0,0,0,0,0], [1,0,0,0,0,0,0,0], [0,0,0,1,0,0,0,0], [0,0,1,0,0,0,0,0], [0,0,0,0,0,1,0,0], [0,0,0,0,1,0,0,0], [0,0,0,0,0,0,0,1], [0,0,0,0,0,0,1,0] ])

a3 = np.add(a2, a2x)

b = np.array([float(1/np.sqrt(8)),float(1/np.sqrt(8)),float(1/np.sqrt(8)),float(1/np.sqrt(8)),float(1/np.sqrt(8)),float(1/np.sqrt(8)),float(1/np.sqrt(8)),float(1/np.sqrt(8))])

print((b.dot(a3.dot(o)/(np.linalg.norm(a3.dot(o))))))

{'000': 8192}
(0.5066062991355671+0j)


In [12]:
print((b.dot(a3.dot(o)/(np.linalg.norm(a3.dot(o))))))

(0.9818942624058683+0j)


In [14]:
def extract_counts(dict):
    total_counts = 0
    relevant_count0 = 0
    relevant_count1 = 0
    for key in dict:
        if key[0] == '1':
            if key[-1] == '1':
                relevant_count1 += int(dict[key])
            if key[-1] == "0":
                relevant_count0 += int(dict[key])
        total_counts += int(dict[key])
    return (relevant_count0/total_counts, relevant_count1/total_counts)

def extract_solution(probs):
    return (np.sqrt(probs[0]), np.sqrt(probs[1]))

def calculate_normalized_fidelity(v1, v2):
    return np.dot(np.array(v1)/np.linalg.norm(v1), np.array(v2)/np.linalg.norm(v2))

In [16]:
c = extract_counts(result.get_counts())
s = extract_solution(c)
print(c, s)

(0.529052734375, 0.4615478515625) (0.7273601132692114, 0.679373131322177)


In [72]:
quito1 = {'000': 348, '001': 255, '010': 222, '011': 275, '100': 2117, '101': 1530, '110': 2048, '111': 1397}
quito2 =  {'000': 350, '001': 247, '010': 189, '011': 258, '100': 2085, '101': 1571, '110': 2067, '111': 1425}
quito3 = {'000': 365, '001': 259, '010': 242, '011': 260, '100': 2164, '101': 1531, '110': 1983, '111': 1388}
quito4 = {'000': 352, '001': 243, '010': 225, '011': 239, '100': 2139, '101': 1603, '110': 2054, '111': 1337}
quito5 = {'000': 315, '001': 282, '010': 209, '011': 267, '100': 2173, '101': 1527, '110': 2018, '111': 1401}
q = [quito1, quito2, quito3, quito4, quito5]

In [73]:
perth1 = {'000': 196, '001': 193, '010': 195, '011': 163, '100': 1937, '101': 1305, '110': 2509, '111': 1694}
perth2 = {'000': 197, '001': 162, '010': 216, '011': 158, '100': 1876, '101': 1339, '110': 2582, '111': 1662}
perth3 =  {'000': 184, '001': 188, '010': 207, '011': 170, '100': 1838, '101': 1289, '110': 2604, '111': 1712}
perth4 = {'000': 155, '001': 200, '010': 211, '011': 168, '100': 1816, '101': 1292, '110': 2597, '111': 1753}
perth5 = {'000': 189, '001': 191, '010': 204, '011': 161, '100': 1964, '101': 1273, '110': 2555, '111': 1655}
p = [perth1, perth2, perth3, perth4, perth5]

In [ ]:
p_avg_f = 0
q_avg_f = 0

for qrun in q:
    probs = extract_counts(qrun)